In [1]:
import pandas as pd
import os
import numpy as np
import zipfile
import matplotlib.pyplot as plt
import graphviz
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import tree
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import files
uploaded = files.upload()

Saving homework.zip to homework.zip


In [3]:
import os
import zipfile
import pandas as pd

# Завдання правильного шляху до zip-файлу
zip_file_path = 'homework.zip'
target_folder = '/content'
os.makedirs(target_folder, exist_ok=True)

# Розпакування zip-архиву
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(target_folder)

# Визначення шляху до розпакованих даних
work_data = os.path.join(target_folder, 'data')

# Ініціалізація порожнього DataFrame для зберігання всіх даних
all_data = pd.DataFrame()

# Обхід усіх папок та файлів у папці `data`
for folder in os.listdir(work_data):
    folder_name = os.path.join(work_data, folder)

    if not os.path.isdir(folder_name):
        continue

    for file in os.listdir(folder_name):
        file_path = os.path.join(folder_name, file)

        if not file.endswith('.csv'):
            continue

        df = pd.read_csv(file_path)
        df['activity'] = folder

        all_data = pd.concat([all_data, df], ignore_index=True)

In [22]:
all_data = all_data.sample(frac=1).reset_index(drop=True)
all_data.head(5)

,accelerometer_X,accelerometer_Y,accelerometer_Z,activity
0,-1.924938,8.082825,-2.537854,running
1,-3.160346,-6.378153,0.474052,walking
2,21.471200,17.372328,5.257667,running
3,-0.766144,4.505888,8.073248,idle
4,4.199430,-10.946290,0.081403,running


In [23]:
all_data.shape

(193860, 4)

In [24]:
X = all_data.drop(['activity'], axis=1)
X

,accelerometer_X,accelerometer_Y,accelerometer_Z
0,-1.924938,8.082825,-2.537854
1,-3.160346,-6.378153,0.474052
2,21.471200,17.372328,5.257667
3,-0.766144,4.505888,8.073248
4,4.199430,-10.946290,0.081403
...,...,...,...
193855,0.296881,-0.019154,9.773131
193856,4.831499,-3.945644,-4.367024
193857,4.319140,9.811439,3.361459
193858,9.327810,-7.225701,-0.421379


In [25]:
y = all_data['activity']

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
# Розрахуємо тимчасові ознаки як середнє за кожним рядком:
X_train['time_mean'] = X_train.mean(axis=1)
X_test['time_mean'] = X_test.mean(axis=1)
X_train

,accelerometer_X,accelerometer_Y,accelerometer_Z,time_mean
186383,8.063671,-7.158663,0.186748,0.363919
85476,-4.261679,-9.916783,-8.777143,-7.651868
91093,2.547431,17.496826,5.779603,8.607953
89073,2.628834,-8.360553,0.234632,-1.832362
124211,-12.310986,-10.754754,-1.857900,-8.307880
...,...,...,...,...
119879,2.341529,-0.995988,10.515334,3.953625
103694,-3.471592,9.753978,10.840945,5.707777
131932,2.571373,-2.422932,-1.225831,-0.359130
146867,-0.445322,5.319916,3.423709,2.766101


In [28]:
X_test

,accelerometer_X,accelerometer_Y,accelerometer_Z,time_mean
171248,0.158017,6.311116,7.637504,4.702212
58417,-1.857900,-7.623138,9.021352,-0.153229
54186,-14.729129,23.338678,-5.889736,0.906604
131711,-1.905785,-8.451532,0.732626,-3.208230
27960,-1.848324,-4.740519,0.579397,-2.003149
...,...,...,...,...
176737,-5.995081,20.077776,5.252879,6.445191
122687,5.726931,6.799533,-2.336741,3.396574
77610,28.218061,25.139118,12.095507,21.817562
133091,0.411803,-0.014365,9.749189,3.382209


In [29]:
# масштабуємо ознаки даних
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train.drop(['time_mean'], axis=1))

X_test_scaled = scaler.transform(X_test.drop(['time_mean'], axis=1))

# Після масштабування даних за допомогою StandardScaler або MinMaxScaler ми отримуємо масив numpy з масштабованими значеннями ознак.
# У цьому рядку ми перетворимо цей масив назад у DataFrame і встановлюємо імена колонок, які були в оригінальних даних,
# за винятком останньої колонки (тобто time_mean), яку ми будемо додавати окремо.
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns[:-1])
# Аналогічно, конвертуємо масштабовані тестові дані в DataFrame і встановлюємо імена колонок, які були в оригінальних даних, за винятком time_mean.
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns[:-1])

# Після масштабування та перетворення ознак у DataFrame, додаємо колонку time_mean назад у тренувальні дані.
# Використовуємо X_train['time_mean'].values, щоб зберегти значення тимчасової ознаки з оригінального DataFrame.
X_train_scaled['time_mean'] = X_train['time_mean'].values
# Аналогічно, додаємо колонку time_mean назад до тестових даних, використовуючи значення з оригінального DataFrame X_test.
X_test_scaled['time_mean'] = X_test['time_mean'].values

In [30]:
import time

model_svc = SVC()

start_time = time.time()
model_svc.fit(X_train_scaled, y_train)
end_time = time.time()

elapsed_time = end_time - start_time
print(f"Час, витрачений на навчання моделі: {elapsed_time:.4f} секунд")

test_accuracy = model_svc.score(X_test_scaled, y_test)
print(f"Точність на тестових даних: {test_accuracy:.4f}")

Час, витрачений на навчання моделі: 608.7030 секунд
Точність на тестових даних: 0.8815


In [31]:
# Виконаємо RFC тренування

import time

# Створення моделі RandomForestClassifier
model_rfs = RandomForestClassifier()

start_time = time.time()
model_rfs.fit(X_train_scaled, y_train)
end_time = time.time()

elapsed_time = end_time - start_time
print(f"Час, витрачений на навчання моделі: {elapsed_time:.4f} секунд")

test_accuracy = model_rfs.score(X_test_scaled, y_test)
print(f"Точність на тестових даних: {test_accuracy:.4f}")

Час, витрачений на навчання моделі: 25.6589 секунд
Точність на тестових даних: 0.9998


In [32]:
# Тестові дані:

start_time = time.time()
svc_prediction = model_svc.predict(X_test_scaled)
end_time = time.time()

elapsed_time = end_time - start_time
print(f"Час, витрачений на передбачення моделі SVM: {elapsed_time:.4f} секунд")

Час, витрачений на передбачення моделі SVM: 101.5460 секунд


In [33]:
# Вимірювання часу виконання передбачень моделі RandomForestClassifier
start_time = time.time()
rfs_prediction = model_rfs.predict(X_test_scaled)
end_time = time.time()

elapsed_time = end_time - start_time
print(f"Час, витрачений на передбачення моделі RandomForestClassifier: {elapsed_time:.4f} секунд")


Час, витрачений на передбачення моделі RandomForestClassifier: 0.6567 секунд


In [34]:
# Зробимо оцінку результатів:
svm_accuracy = accuracy_score(y_test, svc_prediction)
rfc_accuracy = accuracy_score(y_test, rfs_prediction)

print("Accuracy (SVM):", svm_accuracy)
print("Accuracy (Random Forest):", rfc_accuracy)

Accuracy (SVM): 0.881538223460229
Accuracy (Random Forest): 0.9998452491488703


In [35]:
# Цей код створює звіт про те, як модель SVM працює на тестових даних.
# classification_report порівнює правильні відповіді (y_test) з тими, що передбачила модель (svc_prediction).
# Потім він показує, наскільки добре модель вгадала кожен клас.

svm_report = classification_report(y_test, svc_prediction)

print(svm_report)

              precision    recall  f1-score   support

        idle       0.94      0.99      0.96      6319
     running       0.94      0.87      0.91     20468
      stairs       1.00      0.00      0.00       999
     walking       0.76      0.92      0.84     10986

    accuracy                           0.88     38772
   macro avg       0.91      0.70      0.68     38772
weighted avg       0.89      0.88      0.87     38772



In [36]:
# Цей код створює звіт про те, як добре модель Random Forest (RFC) працює на тестових даних.
# classification_report порівнює правильні відповіді (y_test) з прогнозами моделі (rfs_prediction).
# Потім він показує, наскільки добре модель вгадала кожен клас.

rfc_report = classification_report(y_test, rfs_prediction)

print(rfc_report)


              precision    recall  f1-score   support

        idle       1.00      1.00      1.00      6319
     running       1.00      1.00      1.00     20468
      stairs       1.00      0.99      1.00       999
     walking       1.00      1.00      1.00     10986

    accuracy                           1.00     38772
   macro avg       1.00      1.00      1.00     38772
weighted avg       1.00      1.00      1.00     38772



Висновок:

Найбільш точні та кращі результати показав алгоритм RFC, що склав precision = 1
Крім того, RFC навчання за часом тривало в рази менше в порівнянні з SVM